In [5]:
# basically game of life
def get_grid(filepath):
    with open(filepath, 'r') as file:
        grid = [list(row.rstrip()) for row in file.readlines()]
    return grid

In [8]:
class Item:
    # . = open, | = tree, # = lumberyard
    OPEN = '.'
    TREE = '|'
    LUMBERYARD = '#'

In [69]:
def get_neighbour_count(grid, x, y):
    width, height = len(grid[0]), len(grid)
    count = {Item.OPEN: 0, Item.TREE: 0, Item.LUMBERYARD: 0}
    for x_off in (-1, 0, 1):
        for y_off in (-1, 0, 1):
            if x_off == 0 and y_off == 0:
                continue
            x_coord = x+x_off
            y_coord = y+y_off
            if x_coord < 0 or x_coord >= width or\
               y_coord < 0 or y_coord >= height:
                continue
            #print('Adding [{}][{}]:{}'.format(x_coord, y_coord, grid[y_coord][x_coord]))
            count[grid[y_coord][x_coord]] += 1
    return count

def get_next_state(grid, x, y):
    char = grid[y][x]
    count = get_neighbour_count(grid, x, y)
    # if at least 3 trees, open area has trees
    if char == Item.OPEN:
        if count[Item.TREE] >= 3:
            return Item.TREE
        return char
    # if more or equal to 3 lumberyards, no more trees
    elif char == Item.TREE:
        if count[Item.LUMBERYARD] >= 3:
            return Item.LUMBERYARD
        return char
    # lumberyard: at least one lumberyard and tree, otherwise open
    elif char == Item.LUMBERYARD:
        if count[Item.LUMBERYARD] >= 1 and count[Item.TREE] >= 1:
            return char
        return Item.OPEN

def update_grid(grid, n=1):
    width, height = len(grid[0]), len(grid)
    grid     = [[item for item in row] for row in grid]
    buffer   = [[Item.OPEN for _ in range(width)] for _ in range(height)]
    for _ in range(n):
        for y, row in enumerate(grid):
            for x, char in enumerate(row):
                buffer[y][x] = get_next_state(grid, x, y)
        tmp = grid
        grid = buffer
        buffer = tmp
    return grid

In [70]:
def grid_to_string(grid):
    return '\n'.join([''.join(row) for row in grid])

In [75]:
filepath = 'data/day18.txt'
#filepath = 'data/day18_mini.txt'
grid = get_grid(filepath)
#print(grid_to_string(grid))

In [76]:
updated_grid = update_grid(grid, 10)
#print(grid_to_string(updated_grid))

In [77]:
def get_total_resource_value(grid):
    count = {Item.TREE:0, Item.LUMBERYARD:0, Item.OPEN:0}
    for row in grid:
        for char in row:
            count[char] += 1
    return count[Item.TREE] * count[Item.LUMBERYARD]

In [84]:
def update_until_repeat(grid):
    width, height = len(grid[0]), len(grid)
    grid     = [[item for item in row] for row in grid]
    buffer   = [[Item.OPEN for _ in range(width)] for _ in range(height)]
    grid_cache = {}
    total_iter = 0
    while True:
        for y, row in enumerate(grid):
            for x, char in enumerate(row):
                buffer[y][x] = get_next_state(grid, x, y)
        # swap buffers
        total_iter += 1
        tmp = grid
        grid = buffer
        buffer = tmp
        # check if grid already exists
        grid_str = grid_to_string(grid)
        if grid_str in grid_cache:
            break
        grid_cache[grid_str] = total_iter
    return (grid, (grid_cache[grid_str], total_iter))

In [85]:
total_resource_value = get_total_resource_value(updated_grid)
print('Resource value: {}'.format(total_resource_value))

Resource value: 678529


In [86]:
stable_grid, total_repeats = update_until_repeat(grid)

In [88]:
period = total_repeats[1]-total_repeats[0]
print(total_repeats)

(431, 459)


In [90]:
total_minutes = 1000000000

reduced_minutes = (total_minutes-total_repeats[1])%period + total_repeats[0]
print(reduced_minutes)
resource_value = get_total_resource_value(update_grid(grid, reduced_minutes))
print(resource_value)

440
224005
